# 7.3 工具编排与回退

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/07-function-calling/orchestration.ipynb)

**预计 API 费用: ~$0.03**

多工具编排、重试机制、回退策略。

In [ ]:
!pip install -q openai

In [ ]:
import os
from getpass import getpass

if not os.environ.get('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass('OpenAI API Key: ')

## 实验 1: 工具链 (Tool Chain)

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

tools = [
    {'type': 'function', 'function': {'name': 'get_weather', 'description': '获取天气', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string'}}, 'required': ['city']}}},
    {'type': 'function', 'function': {'name': 'suggest_clothing', 'description': '根据温度推荐穿衣', 'parameters': {'type': 'object', 'properties': {'temperature': {'type': 'number'}}, 'required': ['temperature']}}}
]

def get_weather(city: str):
    return {'temperature': 5, 'weather': '晴'}

def suggest_clothing(temperature: float):
    if temperature < 10:
        return {'suggestion': '羽绒服'}
    elif temperature < 20:
        return {'suggestion': '毛衣'}
    return {'suggestion': 'T恤'}

functions = {'get_weather': get_weather, 'suggest_clothing': suggest_clothing}

messages = [{'role': 'user', 'content': '北京天气如何?应该穿什么?'}]

for i in range(5):
    print(f'\n=== 迭代 {i+1} ===')
    response = client.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools)
    message = response.choices[0].message
    messages.append(message)
    
    if not message.tool_calls:
        print(f'✅ {message.content}')
        break
    
    for tc in message.tool_calls:
        name = tc.function.name
        args = json.loads(tc.function.arguments)
        print(f'🔧 {name}({args})')
        result = functions[name](**args)
        print(f'📊 {result}')
        messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': json.dumps(result, ensure_ascii=False)})

## 实验 2: 重试与回退

In [ ]:
import time
import random

class ToolExecutor:
    def __init__(self, max_retries=3):
        self.max_retries = max_retries
    
    def execute_with_retry(self, func, args, fallback_func=None):
        for attempt in range(self.max_retries):
            try:
                result = func(**args)
                if 'error' not in result:
                    return {'success': True, 'data': result}
                raise Exception(result['error'])
            except Exception as e:
                print(f'尝试 {attempt+1} 失败: {e}')
                if attempt < self.max_retries - 1:
                    time.sleep(2 ** attempt)
        
        if fallback_func:
            print('切换到备用工具')
            return {'success': True, 'data': fallback_func(**args), 'fallback': True}
        
        return {'success': False, 'error': str(e)}

def get_weather_api(city: str):
    if random.random() < 0.7:  # 70% 成功率
        raise Exception('API 超时')
    return {'temperature': 15, 'source': 'API'}

def get_weather_cache(city: str):
    return {'temperature': 15, 'source': 'cache'}

executor = ToolExecutor()
result = executor.execute_with_retry(get_weather_api, {'city': '北京'}, get_weather_cache)
print(f'\n结果: {result}')

## 关键要点

1. **工具链**: 多轮对话实现顺序调用
2. **重试机制**: 指数退避
3. **回退策略**: 主工具失败后切换备用
4. **max_iterations**: 防止死循环

---

**下一步**: 学习第 8 章多模态 AI